In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

c:\Users\jaesc2\GitHub\skforecast


In [104]:
import keras
import numpy as np
import pandas as pd

from skforecast.deep_learning import ForecasterRnn
from skforecast.deep_learning.utils import create_and_compile_model

series = pd.DataFrame(
    {
        "1": pd.Series(np.arange(50.)),
        "2": pd.Series(np.arange(50., 100.)),
        "3": pd.Series(np.arange(100., 150.)),
    }
)
lags = 3
steps = 4
levels = ["1", "2"]
activation = "relu"
optimizer = keras.optimizers.Adam(learning_rate=0.01)
loss = keras.losses.MeanSquaredError()
recurrent_units = 100
dense_units = [128, 64]


model = create_and_compile_model(
    series=series,
    lags=lags,
    steps=steps,
    levels=levels,
    recurrent_units=recurrent_units,
    dense_units=dense_units,
    activation=activation,
    optimizer=optimizer,
    loss=loss,
)

keras version: 3.6.0
Using backend: tensorflow
tensorflow version: 2.18.0


In [119]:
forecaster = ForecasterRnn(model, levels, lags=lags)
forecaster.fit(series)

c:\Users\jaesc2\Miniconda3\envs\skforecast_py11_2\Lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 20 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.34071


In [120]:
forecaster.series_names_in_

['1', '2', '3']

In [121]:
forecaster.levels

['1', '2']

In [122]:
forecaster.last_window_

,1,2,3
47,47.0,97.0,147.0
48,48.0,98.0,148.0
49,49.0,99.0,149.0


In [123]:
inputs = forecaster._create_predict_inputs(steps=3, levels=['1', '2'])
inputs

(array([[[0.95918367, 0.95918367, 0.95918367],
         [0.97959184, 0.97959184, 0.97959184],
         [1.        , 1.        , 1.        ]]]),
 [],
 [1, 2, 3],
 ['1', '2'],
 Index([50, 51, 52], dtype='int64'))

In [124]:
predictions = forecaster.predict(steps=3, levels=['1'])
predictions

,level,pred
50,1,7.713717
51,1,25.941444
52,1,31.411823


In [125]:
predictions = forecaster.predict(steps=3, levels=None)
predictions

,level,pred
50,1,7.713717
50,2,61.483234
51,1,25.941444
51,2,55.504932
52,1,31.411823
52,2,82.724594


In [141]:
predictions = forecaster._predict_interval_conformal(steps=3, levels=None)
predictions

[[0.75102472 0.68118361]
 [0.44017978 0.8132577 ]
 [0.3597596  0.33670886]]


,level,pred,lower_bound,upper_bound
50,1,7.713717,-29.086494,44.513928
50,2,61.483238,28.105241,94.861235
51,1,25.941444,4.372634,47.510253
51,2,55.504931,15.655304,95.354558
52,1,31.411823,13.783603,49.040043
52,2,82.724598,66.225864,99.223332


In [140]:
predictions = forecaster._predict_interval_conformal(steps=3, levels=['1'])
predictions

,level,pred,lower_bound,upper_bound
50,1,7.713717,-29.086494,44.513928
51,1,25.941444,4.372634,47.510253
52,1,31.411823,13.783603,49.040043


In [144]:
np.quantile(
    np.abs(forecaster.in_sample_residuals_[2][:, 1]), 0.95
)

np.float64(0.8132576974450934)

In [ ]:
forecaster.in_sample_residuals_[1]

{1: array([[ 0.00300063, -0.02063499],
        [ 0.02164905, -0.00324193],
        [ 0.04019636,  0.01412207],
        [ 0.05871693,  0.03156052],
        [ 0.07703444,  0.04902489],
        [ 0.09533564,  0.06641259],
        [ 0.11370614,  0.0837122 ],
        [ 0.13206353,  0.10091294],
        [ 0.15043704,  0.11811714],
        [ 0.1688066 ,  0.1353427 ],
        [ 0.1871703 ,  0.15255236],
        [ 0.20552565,  0.16975107],
        [ 0.2238739 ,  0.18696467],
        [ 0.2422113 ,  0.20416079],
        [ 0.26053225,  0.22133179],
        [ 0.2789068 ,  0.23851671],
        [ 0.29727427,  0.2556899 ],
        [ 0.31563729,  0.27285653],
        [ 0.33399589,  0.29001659],
        [ 0.35235008,  0.30717007],
        [ 0.37069992,  0.32431521],
        [ 0.38904568,  0.34144994],
        [ 0.40738741,  0.3585762 ],
        [ 0.42566772,  0.37572525],
        [ 0.44391787,  0.39288245],
        [ 0.46216279,  0.41003336],
        [ 0.48040247,  0.42717802],
        [ 0.49863687,  0.